### LOADING DATA 

In [1]:
#Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
%matplotlib inline

In [22]:
#Importing the Data
df=pd.read_csv('OneDrive\Desktop\Projects\WinNLP\AmazonRev.csv')
df.head()

C:\Users\smrit\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [23]:
#Checking for null values in relevant columns
perm=df[['reviews.rating' , 'reviews.text' , 'reviews.title' , 'reviews.username']]
print(perm.isnull().sum())
perm.head()

reviews.rating      33
reviews.text         1
reviews.title        5
reviews.username     2
dtype: int64


,reviews.rating,reviews.text,reviews.title,reviews.username
0,5.0,This product so far has not disappointed. My c...,Kindle,Adapter
1,5.0,great for beginner or experienced person. Boug...,very fast,truman
2,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,DaveZ
3,4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!,Shacks
4,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,explore42


In [24]:
#Obtaining null values in relevant columns
check=df[df["reviews.rating"].isnull()]
check.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
2886,AVpfBEWcilAPnD_xTGb7,"Kindle Keyboard,,,\r\nKindle Keyboard,,,",B002Y27P3M,Amazon,"Kindle Store,Amazon Devices,Electronics","kindlekeyboard/b002y27p3m,amazon/d01101",Amazon,NaN,NaN,2015-12-14T00:00:00Z,...,NaN,NaN,NaN,NaN,http://www.amazon.com/Kindle-Wireless-Reader-W...,The Kindle is my first e-ink reader. I own an ...,"Worth the money. Not perfect, but very very go...",NaN,NaN,Jeffrey Stanley
2887,AVpfBEWcilAPnD_xTGb7,"Kindle Keyboard,,,\r\nKindle Keyboard,,,",B002Y27P3M,Amazon,"Kindle Store,Amazon Devices,Electronics","kindlekeyboard/b002y27p3m,amazon/d01101",Amazon,NaN,NaN,2015-10-19T00:00:00Z,...,NaN,NaN,NaN,NaN,http://www.amazon.com/Kindle-Wireless-Reader-W...,"I'm a first-time Kindle owner, so I have nothi...","I Wanted a Dedicated E-Reader, and That's What...",NaN,NaN,Matthew Coenen
2888,AVpfBEWcilAPnD_xTGb7,"Kindle Keyboard,,,\r\nKindle Keyboard,,,",B002Y27P3M,Amazon,"Kindle Store,Amazon Devices,Electronics","kindlekeyboard/b002y27p3m,amazon/d01101",Amazon,NaN,NaN,2015-12-08T00:00:00Z,...,NaN,NaN,NaN,NaN,http://www.amazon.com/Kindle-Wireless-Reader-W...,UPDATE NOVEMBER 2011:My review is now over a y...,Kindle vs. Nook (updated),NaN,NaN,Ron Cronovich
2889,AVpfBEWcilAPnD_xTGb7,"Kindle Keyboard,,,\r\nKindle Keyboard,,,",B002Y27P3M,Amazon,"Kindle Store,Amazon Devices,Electronics","kindlekeyboard/b002y27p3m,amazon/d01101",Amazon,NaN,NaN,2015-11-30T00:00:00Z,...,NaN,NaN,NaN,NaN,http://www.amazon.com/Kindle-Wireless-Reader-W...,"I'm a first-time Kindle owner, so I have nothi...","I Wanted a Dedicated E-Reader, and That's What...",NaN,NaN,Matthew Coenen
2890,AVpfBEWcilAPnD_xTGb7,"Kindle Keyboard,,,\r\nKindle Keyboard,,,",B002Y27P3M,Amazon,"Kindle Store,Amazon Devices,Electronics","kindlekeyboard/b002y27p3m,amazon/d01101",Amazon,NaN,NaN,2015-10-09T00:00:00Z,...,NaN,NaN,NaN,NaN,http://www.amazon.com/Kindle-Wireless-Reader-W...,I woke up to a nice surprise this morning: a n...,"Not the perfect do-it-all device, but very clo...",NaN,NaN,C. Tipton


In [61]:
#obtaining non null values in relevant columns
senti=perm[perm["reviews.rating"].notnull()]
perm.head()

,reviews.rating,reviews.text,reviews.title,reviews.username
0,5.0,This product so far has not disappointed. My c...,Kindle,Adapter
1,5.0,great for beginner or experienced person. Boug...,very fast,truman
2,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,DaveZ
3,4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!,Shacks
4,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,explore42


In [62]:
#Classification of text as positive and negative
senti["senti"]=senti["reviews.rating"]>=4
senti["senti"]=senti["senti"].replace([True , False] , ["pos" , "neg"])

C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### PREPROCESSING TEXT TO TRAIN MODEL

In [64]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

cleanup_re=re.compile('[^a-z]+')
def cleanup(sentence):
    sentence=str(sentence)
    sentence=sentence.lower()
    sentence=cleanup_re.sub(' ', sentence).strip()
    sentence=" ".join(nltk.word_tokenize(sentence))
    return sentence

senti["Summary_Clean"]=senti["reviews.text"].apply(cleanup)
check["Summary_Clean"]=check["reviews.text"].apply(cleanup)

C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [66]:
#Splitting data into train and test set
split=senti[["Summary_Clean","senti"]]
train=split.sample(frac=0.8,random_state=200)
test=split.drop(train.index)

In [68]:
#Feature Extractor 
def word_feats(words):
    features={}
    for word in words:
        features[word]=True
    return features

train["words"]=train["Summary_Clean"].str.lower().str.split()
test["words"]=test["Summary_Clean"].str.lower().str.split()
check["words"]=check["Summary_Clean"].str.lower().str.split()
train.index=range(train.shape[0])
test.index=range(test.shape[0])
check.index=range(check.shape[0])
prediction={}
train_naive = []
test_naive = []
check_naive = []

for i in range(train.shape[0]):
    train_naive = train_naive +[[word_feats(train["words"][i]) , train["senti"][i]]]
for i in range(test.shape[0]):
    test_naive = test_naive +[[word_feats(test["words"][i]) , test["senti"][i]]]
for i in range(check.shape[0]):
    check_naive = check_naive +[word_feats(check["words"][i])]
    
classifier = NaiveBayesClassifier.train(train_naive)
print("NLTK Naive bayes Accuracy : {}".format(nltk.classify.util.accuracy(classifier , test_naive)))
classifier.show_most_informative_features(5)

C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


NLTK Naive bayes Accuracy : 0.5885920577617328
Most Informative Features
                 deleted = True              neg : pos    =     51.3 : 1.0
                 warning = True              neg : pos    =     51.3 : 1.0
                  rotate = True              neg : pos    =     42.0 : 1.0
                    nope = True              neg : pos    =     42.0 : 1.0
                    bent = True              neg : pos    =     42.0 : 1.0


In [69]:
#Predicting result of NLTK Classifier
y=[]
only_words=[test_naive[i][0] for i in range(test.shape[0])]
for i in range(test.shape[0]):
    y=y+[classifier.classify(only_words[i] )]
prediction["Naive"]= np.asarray(y)

y1=[]
for i in range(check.shape[0]):
    y1=y1+[classifier.classify(check_naive[i] )]
check["Naive"] = y1

C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [70]:
#Building TF-IDF Vectors and Count Vectors for data

from wordcloud import STOPWORDS
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
stopwords=set(STOPWORDS)
stopwords.remove("not")

count_vect=CountVectorizer(min_df=2 ,stop_words=stopwords , ngram_range=(1,2))
tfidf_transformer=TfidfTransformer()

X_train_counts=count_vect.fit_transform(train["Summary_Clean"])        
X_train_tfidf=tfidf_transformer.fit_transform(X_train_counts)

X_new_counts=count_vect.transform(test["Summary_Clean"])
X_test_tfidf=tfidf_transformer.transform(X_new_counts)

checkcounts=count_vect.transform(check["Summary_Clean"])
checktfidf=tfidf_transformer.transform(checkcounts)

C:\Users\smrit\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [71]:
#Using Logistic Regression to classify 
from sklearn import linear_model
logreg=linear_model.LogisticRegression(solver='lbfgs' , C=1000)
logistic=logreg.fit(X_train_tfidf, train["senti"])
prediction['LogisticRegression']=logreg.predict_proba(X_test_tfidf)[:,1]
print("Logistic Regression Accuracy : {}".format(logreg.score(X_test_tfidf , test["senti"])))

check["log"]=logreg.predict(checktfidf)

Logistic Regression Accuracy : 0.9374729241877257


C:\Users\smrit\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\smrit\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [72]:
#Obtaining Most Commonly Occuring Words
words=count_vect.get_feature_names()
feature_coefs=pd.DataFrame(data = list(zip(words, logistic.coef_[0])),columns = ['feature', 'coef'])
feature_coefs.sort_values(by="coef")

,feature,coef
42500,terrible,-20.116774
48077,will definitely,-19.293255
10643,done great,-19.260647
38824,slow,-18.697867
18441,great year,-17.490821
36318,returning,-17.290554
28288,not,-17.059630
32534,price isn,-16.207469
45908,using firestick,-16.198127
16573,gift three,-15.867423


In [79]:
def test_sample(model, sample):
    sample_counts = count_vect.transform([sample])
    sample_tfidf = tfidf_transformer.transform(sample_counts)
    result = model.predict(sample_tfidf)[0]
    prob = model.predict_proba(sample_tfidf)[0]
    print("Sample estimated as %s: negative prob %f, positive prob %f" % (result.upper(), prob[0], prob[1]))

test_sample(logreg, "The product was good and easy to  use")
test_sample(logreg, "the whole experience was horrible and product is worst")
test_sample(logreg, "product is not good")
test_sample(logreg, "Bad product value.")
test_sample(logreg, "I'm really impressed with the quality!")

Sample estimated as POS: negative prob 0.000000, positive prob 1.000000
Sample estimated as NEG: negative prob 0.993632, positive prob 0.006368
Sample estimated as NEG: negative prob 0.965476, positive prob 0.034524
Sample estimated as NEG: negative prob 0.970391, positive prob 0.029609
Sample estimated as POS: negative prob 0.000013, positive prob 0.999987
